 # Pridobivanje globalnih podnebnih podatkov

## 1. Uvod


 ## 2. Nastavitev okolja
 ### 2.1 Uvoz potrebnih knjižnic

-  Pri pogonu ( `Run All` ) se avtomatsko naložijo vse potrebne knjižnice.

-  V primeru napake, je treba prvo pognati funkcijo `zahteve_namestitev.py`, potem pa v tem notebooku `Restart Kernel` in ponovno `Run All`

In [1]:
from zahteve_namestitev import instalacija
instalacija()

from datetime import datetime           # Datumi
import pandas as pd                     # Pandas
from tqdm import tqdm                   # Progress bar
from pathlib import Path                # Poti do datotek
import warnings                         # Prikazovanje težav
import os                               # Operacijski sistem
import logging                          # Shranjevanje in prikazovanje težav
from meteostat import Point, Daily      # Geografska lokacija in dnevni podatki o temperaturi


 ### 2.2 Nastavimo poti in ustvarimo mape
-  Nastavimo osnovne poti do map v uporabi,

-  `.mkdir()` - Preveri obstoj potrebnih map in jih v primeru, da še ne obstajajo naredi;

-  Nastavimo spremenljivki za začetni in končni datum,

-  Preverimo lokacijo vseh map.

In [2]:
Glavna_mapa = Path.cwd()
Podatkovna_baza = Glavna_mapa / "Podatki" / "nepredelani"
Rezultati = Glavna_mapa / "Podatki" / "rezultati"

Podatkovna_baza.mkdir(parents=True, exist_ok=True)
Rezultati.mkdir(parents=True, exist_ok=True)

zacetek = datetime(1950, 1, 1)
konec = datetime(2024, 12, 31)

print(f"""
Glavni imenik:      {Glavna_mapa}
Mapa za podatke:    {Podatkovna_baza}
Mapa za rezultate:  {Rezultati} """)


Glavni imenik:      c:\Users\jan\Projektna_naloga_UVP
Mapa za podatke:    c:\Users\jan\Projektna_naloga_UVP\Podatki\nepredelani
Mapa za rezultate:  c:\Users\jan\Projektna_naloga_UVP\Podatki\rezultati 


 ### 2.3 Pomožne skripte
-  `ustvari_mapo()` - ustvari novo mapo na podani poti;

-  `shrani_podatke()` - datoteko v formatu DataFrame pretvori v CSV in shrani pod imenom: 
   
   `[ime]_[leto]_temperature.csv`

In [3]:
def ustvari_mapo(pot):

    os.makedirs(pot, exist_ok=True)

def shrani_podatke(df, lokacija_ime, leto):

    pot = Podatkovna_baza / lokacija_ime
    ustvari_mapo(pot)
    datoteka = pot / f"{lokacija_ime}_{leto}_temperature.csv"
    df.to_csv(datoteka, index=False)

 ### 2.4 Lokacije
-  Definiramo izbrane lokacije v slovarju,

-  Poleg imena jim dodamo njihovo geografsko lokacijo.

   ( latitude / `lat` = širina,  longitude / `lon` = dolžina )"

In [4]:
lokacije = [
    # Severna vzhodna polobla
    {'ime': 'Tokyo', 'lat': 35.68, 'lon': 139.77},
    {'ime': 'Berlin', 'lat': 52.52, 'lon': 13.41},
    {'ime': 'Reykjavik', 'lat': 64.13, 'lon': -21.82},
    {'ime': 'Prag', 'lat': 50.09, 'lon': 14.42},     

    # Severna zahodna polobla
    {'ime': 'Alert', 'lat': 82.50, 'lon': -62.33},
    {'ime': 'Denver', 'lat': 39.74, 'lon': -104.99},
    {'ime': 'Toronto', 'lat': 43.70, 'lon': -79.42},

    # Južna vzhodna polobla
    {'ime': 'Sydney', 'lat': -33.87, 'lon': 151.21},
    {'ime': 'McMurdo_Station', 'lat': -77.85, 'lon': 166.67},
    {'ime': 'Auckland', 'lat': -36.85, 'lon': 174.76},
    
    # Južna zahodna polobla
    {'ime': 'Santiago', 'lat': -33.45, 'lon': -70.67},
    {'ime': 'Lima', 'lat': -12.05, 'lon': -77.03},
]


 ## 3 Pridobivanje podatkov

-  `.filterwarnings()` in `.getLogger()` onemogočita pošiljanje prekomernih warningov

-  Definiramo funkcijo `pridobi_podatke_za_lokacije`, ki pridobi podatke za vse
   definirane lokacije v določenem časovnem obdobju.

-  Zbiranje podatkov za vsako lokacijo se kaže preko progress bara (paket `tqdm`).

-  V zanki `Try`, ki zajema morebitne napake, prvo nastavimo spremenljivki:
    
    - `tocka` na geografsko lokacijo kraja s funkcijo `Point()`
       in `podatki` na vse dostopne dnevne informacije o vremenu s funkcijo `Daily()`.

    - Funkcija `fetch()` nato pošlje zahtevo k Meteostat API in podatke dobi v obliki `df`
       (DataFrame).

    - Preoblikujemo DataFrame in nastavimo zahtevane stolpce. Nato preverimo obstoj stolpcev in jih po
       potrebi ustvarimo. Če manjka podatek, dodamo v DataFrame `None`.

    -  Če v kateri vrstici manjka podatek za povprečno temperaturo (`tavg`), ga izračunamo iz minimaln
       (`tmin`) in maksimalne temperature (`tmax`).

    -  Vse podatke zaokrožimo na 1 decimalko natančno in manjkajoče podatke nadomestimo s praznimi stringi.

    -  S funkcijo `shrani_podatke()` v Dataframe shranimo podatke po dnevih za eno leto.

-  Kličemo `pridobi_podatke_za_lokacije()`, ki shrani rezultate pridobivanja podatkov v
   spremenljivki `uspeh` in `neuspeh`.

In [5]:
# Onemogočanje sporočil (warnings)
warnings.filterwarnings('ignore')
logging.getLogger('meteostat').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)


def pridobi_podatke_za_lokacije(lokacije, zacetek, konec):

    uspeh = 0
    neuspeh = 0

    with tqdm(lokacije, 
              desc=" Pridobivanje podatkov :",
              bar_format="{l_bar}{bar:30}[{remaining}]"
              ) as pbar:
        for lokacija in pbar:
            pbar.set_description(f"{lokacija['ime'][:15]:<15}") 
            
            try:
                tocka = Point(lokacija['lat'], lokacija['lon'])
                pbar.set_description(f" {lokacija['ime'].ljust(15)}")

                podatki = Daily(tocka, zacetek, konec)
                df = podatki.fetch()

                if df is None or df.empty:
                    neuspeh += 1
                    continue
                
                df = df.reset_index()
                stolpci = ['time', 'tavg', 'tmin', 'tmax', 'prcp', 'snow']
                
                # Preverimo/ustvarimo stolpce:
                for stolpec in stolpci:
                    if stolpec not in df.columns:
                        if stolpec == 'time':
                            continue
                        df[stolpec] = None
                
                # Izračun tavg:
                if 'tmin' in df.columns and 'tmax' in df.columns:
                    for index, row in df.iterrows():
                        if (pd.notnull(row['tmin']) and pd.notnull(row['tmax']) and 
                            ('tavg' not in df.columns or pd.isnull(row['tavg']))):

                            tavg_value = (row['tmin'] + row['tmax']) / 2

                            if 'tavg' not in df.columns:
                                df['tavg'] = None

                            df.at[index, 'tavg'] = tavg_value

                # Zaokrožimo:
                for stolpec in ['tavg', 'tmin', 'tmax', 'prcp', 'snow']:
                    if stolpec in df.columns:
                        df[stolpec] = df[stolpec].apply(lambda x: round(x, 1) if isinstance(x, (int, float)) and not pd.isnull(x) else x)

                # Manjkajoči podatki:
                for stolpec in ['tavg', 'tmin', 'tmax', 'prcp', 'snow']:
                    if stolpec in df.columns:
                        df[stolpec] = df[stolpec].apply(lambda x: '' if pd.isnull(x) else x)
                
                df = df[stolpci].copy()
                
                # Shrani podatke po letih
                leta = df['time'].dt.year.unique()
                with tqdm(leta, desc=f"Shranjujem {lokacija['ime']}", leave=False) as year_pbar:
                    for leto in year_pbar:
                        df_leto = df[df['time'].dt.year == leto].copy()
                        
                        has_data = False
                        for stolpec in ['tavg', 'tmin', 'tmax', 'prcp', 'snow']:
                            if stolpec in df_leto.columns and not all(str(x).strip() == '' for x in df_leto[stolpec] if x is not None):
                                has_data = True
                                break
                        
                        if has_data:
                            shrani_podatke(df_leto, lokacija['ime'], leto)
                
                uspeh += 1

            except Exception as e:

                if "No data available" in str(e):
                    neuspeh += 1
                    continue

                print(f"Napaka pri {lokacija['ime']}: {str(e)}")

                neuspeh += 1
        return uspeh, neuspeh

uspeh, neuspeh = pridobi_podatke_za_lokacije(lokacije, zacetek, konec)

print(f"""
Uspešno:  {uspeh} lokacij
Neuspešno: {neuspeh} lokacij
Mapa s podatki: {Podatkovna_baza} """)


 Lima           : 100%|██████████████████████████████[00:00]    


Uspešno:  12 lokacij
Neuspešno: 0 lokacij
Mapa s podatki: c:\Users\jan\Projektna_naloga_UVP\Podatki\nepredelani 
